# Synthetic dataset generation using RAFT

This recipe will walk you through using a big LLM such as [OpenAI GPT-4o](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models?tabs=python-secure#gpt-4o-and-gpt-4-turbo) or [Meta Llama 3.1 405B](https://aka.ms/c/model/Meta-Llama-3.1-405B-Instruct) deployed on Azure AI to generate a synthetic dataset using UC Berkeley's Gorilla project RAFT method (see [blog post](https://aka.ms/raft-blog)).

## Overview
![](./doc/raft-process-gen.png)

## What is RAFT?

RAFT stands for Retrieval Augmented Fine Tuning. The general principle is to use a big LLM such as Llama 3.1 405B to analyse a set of documents and generate a dataset of questions and answers that users might want to ask about those documents. We can then use that QA dataset to fine tune a smaller model such as Llama 3.1 8B. The fine tune model will therefore be better at answering questions about those documents.

### Analogy: How to prepare a LLM for an Exam? 📝

*Note: This description was copied from the [UC Berkeley RAFT blog post](https://aka.ms/raft-blog-ucb).*

RAFT is a general recipe to finetune a pretrained LLM to your domain-specific RAG settings. This is a common scenario where you want your LLM to answer questions grounded on a set of documents, for e.g., private files in an enterprise. Such a setting is different from the general RAG where the LLM does not know which domain (of documents) it will be tested on. To better illustrate this setting, let's draw an analogy between deploying and using an LLM with the real-world setting of prepararing for an exam.

![RAFT Open book principle](./doc/raft_openbook.png "RAFT Open book principle")

#### Closed-Book Exam

A closed book exam often refers to the scenario where the LLMs do not have access to any additional documents or references to answer the questions during the exam. For LLMs, this is equivalent to the scenario, for example, in which the LLM is used as a chatbot. In this scenario the LLM draws from the knowledge baked in during pre-training and supervised-finetuning to respond to the users' prompt.

#### Open-Book Exam

In contrast, we liken the open-book exam setting to the scenario in which the LLM can refer to external sources of information (e.g., a website or a book chapter). In such scenarios, typically, the LLM is paired with retriever which retrieves k documents (or specific segments of the document) which are appended to the users' prompt. It is only through these documents retrieved that the LLM gains access to new knowledge. As a result, we argue that the LLM's performance in these settings, where it is trained as a general-purpose LLM is largely dependent on the quality of the retriever and how accurately the retriever can identify the most relevant piece of information.

#### RAFT

RAFT focuses on a narrower but increasingly popular domain than the general open book exam, called the domain-specific open-book exam. In domain-specific open book exam, we know a priori the domain in which the LLM will be tested --- used for inference. The LLM can respond to the users' prompt using use any and all information from this specific domain, which it has been fine-tuned on. Examples of domain specific examples include enterprise documents, latest news, code repositories belonging to an organization, etc. In all these scenarios, the LLM will be used to respond to the questions, whose answers can be found within a collection of documents (a small practical domain). The retrieval technique itself has little to no-impact on the mechanism (though it may impact the accuracy). This paper mainly studies this, domain-specific open-book setting and how to adapt a pretrained LLM to this specific domain, including how to make it more robust to a varying number of retrieved documents and distractors.

### RAFT Process: from domain documents to Q/A/CoT dataset splits

The process is the following. RAFT takes as input a set of documents, split them into chunks, and for each chunk generates a list of questions, Chain Of Thought answers with a selection of relevant and irrelevant context chunks.

![RAFT](./doc/raft.png "RAFT")

## Running time and cost

The RAFT script usually takes a few minutes on the default sample document but can take days on bigger domains depending on the number and size of documents and the number of questions being generated for each chunk.

The cost of running this RAFT script on the sample document should be a few dollars. But beware, running it on bigger domains can cost hundreds of dollars if not more. It is safe to run this notebook multiple times though as the costly part, running the `raft.py` script, will only be executed if the dataset doesn't exist yet.

## Pre-requisites

Before running this notebook, let's make sure your environment is ready

### 1. Deploy Meta Llama 3.1 405B Instruct as a serverless endpoint.

This model will be used to generate the synthetic dataset.

You can either use [Azure ML Studio](https://aka.ms/raft-llama-31-learn-deploy-405b) or [Azure AI Studio](https://aka.ms/raft-llama-31-learn-deploy-405b-ai-studio).

**Note**: an Azure ML Workspace is the same as a Azure AI Hub, you will be able to go back and forth between the two transparently.

### 2. Deploy OpenAI's `text-embedding-ada-002` as a serverless endpoint.

This model will be used to create the chunk embeddings.

You can follow the same procedure as for the Meta Llama model deployment

### 3. Setup your environment variables

Copy the `.env.sample` file to `.env` and update according to your Azure AI project configuration and deployed endpoints

## Setup the RAFT repository
 
This script will checkout a shallow and narrow clone of the UC Berkeley Gorilla RAFT repository locally so that this notebook can invoke the RAFT script and util functions. It can safely be run multiple times.

In [4]:
! ./setup_raft.sh

Setup Gorilla RAFT in /workspaces/aitour26-ft-2/src/demo-raft/notebooks/.gorilla
Your branch is up to date with 'origin/raft-distillation-recipe'.
Already up to date.


## Install requirements

The requirements should have been automatically installed if you opened the project in Dev Container or Codespaces, but if not, uncomment the following cell to install the requirements

In [5]:
#! pip install -r requirements.txt

## Check instrastructure requirements

The following integration tests check that the endpoints are accessible and working before executing the RAFT program

In [2]:
! python -m pytest --rootdir=infra/tests/

============================= test session starts ==============================
platform linux -- Python 3.12.8, pytest-8.1.2, pluggy-1.6.0
rootdir: /workspaces/aitour26-ft-2/src/demo-raft/notebooks/infra/tests
configfile: ../../pyproject.toml
plugins: anyio-4.10.0, langsmith-0.4.17
collected 4 items                                                              

infra/tests/test_baseline.py .                                           [ 25%]
infra/tests/test_embeddings.py .                                         [ 50%]
infra/tests/test_scoring.py .                                            [ 75%]
infra/tests/test_teacher.py .                                            [100%]

=============================== warnings summary ===============================
test_baseline.py: 3 warnings
test_embeddings.py: 3 warnings
test_scoring.py: 3 warnings
test_teacher.py: 3 warnings
  /home/vscode/.local/lib/python3.12/site-packages/msal/token_cache.py:182: DeprecationWarning: Use list(search(...)

## Load infrastructure environnment variables

In [1]:
from os import getenv
from dotenv import load_dotenv
from dotenv_azd import load_azd_env
from utils import get_env_state_file

# Load AZD env
load_azd_env(quiet=True)

# User provided values
load_dotenv(".env")

# Variables passed by previous notebooks
load_dotenv(get_env_state_file())

EMBEDDING_DEPLOYMENT_NAME=getenv("EMBEDDING_DEPLOYMENT_NAME")
TEACHER_DEPLOYMENT_NAME=getenv("TEACHER_DEPLOYMENT_NAME")

print(f"Using embedding model {EMBEDDING_DEPLOYMENT_NAME}")
print(f"Using teacher model {TEACHER_DEPLOYMENT_NAME}")

Using embedding model openai-text-embedding-ada-002
Using teacher model openai-gpt-4-1


## Select the documents

#### Notebook parameters

*Note: Parameters are typed as indicated for Papermill introspection*

In [2]:
ds_name: str = "zava-articles"
doc_path: str = "sample_data/zava-articles"
format: str = "chat"
finetuning_train_split : int = .8
finetuning_valid_split : int = .1
finetuning_threshold : int = 369 #65
raft_questions: int = 2

In [3]:
import pandas as pd
from utils import update_state

ds_path = f"dataset/{ds_name}"
ds_output_file = f"{ds_path}.jsonl"
update_state("DATASET_NAME", ds_name)
print("Creating dataset: " + ds_name)

Updating state file .env.state.gpt41 with DATASET_NAME=zava-articles
Creating dataset: zava-articles


In [4]:
! mkdir -p $ds_path

### Overview of PDF

In [8]:
from utils import get_pdf_image
from pathlib import Path

pdf_image = None
if Path(doc_path).exists() and Path(doc_path).is_file() and Path(doc_path).suffix == ".pdf":
    pdf_image = get_pdf_image(doc_path)
pdf_image

### Generate Q/A/CoT fine-tuning dataset using RAFT from the domain specific documents

The `--completion_model` and `--embedding_model` parameters refer to the names of the deployments of the models in Azure.

**Note about `--qa-threshold`**: The Azure AI Finetuning service requires a minimum of 65 samples in the training split so in order to make this notebook run as quickly as possible with a demo dataset, we calculate the minimum number of samples we need to generate overall using the `finetuning_threshold` and the `finetuning_train_split`.

In [5]:
from math import ceil
qa_threshold = ceil(finetuning_threshold / finetuning_train_split)
print(f"QA threshold: {qa_threshold}")

QA threshold: 462


In [6]:
%%bash --out azure_env_name --err null
azd env get-value AZURE_ENV_NAME || echo ""

In [7]:
azure_env_name = azure_env_name.strip()
azure_env_file = f".azure/{azure_env_name}/.env" if azure_env_name else ""
azure_env_file

'.azure/cvi-brk443-2/.env'

In [38]:
! touch .env
! env $(cat .env .env.state $azure_env_file) python3 .gorilla/raft/raft.py \
    --datapath "$doc_path" \
    --output $ds_path \
    --distractors 3 \
    --doctype pdf \
    --chunk_size 512 \
    --questions $raft_questions \
    --workers 2 \
    --system-prompt-key gpt \
    --completion_model $TEACHER_DEPLOYMENT_NAME \
    --embedding_model $EMBEDDING_DEPLOYMENT_NAME \
    --qa-threshold $qa_threshold \
    --completion-env-prefix TEACHER

env #PROJECT_ENDPOINT=https://cvi-brk443-manual-resource.services.ai.azure.com/api/projects/cvi-brk443-manual #ENDPOINT_URL=https://cvi-brk443-manual-resource.openai.azure.com/ #DEPLOYMENT_NAME=gpt-4.1-nano #AZURE_AI_SEARCH_ENDPOINT=https://cvi-brk443-search.search.windows.net #AZURE_AI_SEARCH_INDEX=zava-paint-articles-distractors-v3 BASELINE_AZURE_OPENAI_DEPLOYMENT=openai-gpt-4-1-nano BASELINE_AZURE_OPENAI_ENDPOINT=https://aoai-rogteyz-cvi-brk443-2.openai.azure.com/ BASELINE_DEPLOYMENT_PLATFORM=openai BASELINE_OPENAI_API_VERSION=2023-07-01-preview DATASET_NAME=zava-articles EMBEDDING_AZURE_OPENAI_DEPLOYMENT=openai-text-embedding-ada-002 EMBEDDING_AZURE_OPENAI_ENDPOINT=https://aoai-rogteyz-cvi-brk443-2.openai.azure.com/ EMBEDDING_DEPLOYMENT_PLATFORM=openai EMBEDDING_OPENAI_API_VERSION=2023-07-01-preview JUDGE_AZURE_OPENAI_DEPLOYMENT=openai-gpt-4-1 JUDGE_AZURE_OPENAI_ENDPOINT=https://aoai-rogteyz-cvi-brk443-2.openai.azure.com/ JUDGE_DEPLOYMENT_PLATFORM=openai JUDGE_OPENAI_API_VERSION=20

*Note*: The bit of shell logic wrapping the python script call allows to skip the generation if the dataset has already been generated so it is safe to run this notebook multiple times.

## Prepare training, validation and evaluation splits

Let's define variables for the different files we will need throughout this notebook

In [8]:
raft_arrow_file = f"{ds_path}/data-00000-of-00001.arrow"
dataset_path = f"{ds_path}-files/{ds_name}-full.jsonl"
dataset_path_hf = f"{ds_path}-files/{ds_name}-hf.full.jsonl"

dataset_path_hf_train = f"{ds_path}-files/{ds_name}-hf.train.jsonl"
dataset_path_hf_valid = f"{ds_path}-files/{ds_name}-hf.valid.jsonl"
dataset_path_hf_eval = f"{ds_path}-files/{ds_name}-hf.eval.jsonl"

dataset_path_ft_train = f"{ds_path}-files/{ds_name}-ft.train.jsonl"
dataset_path_ft_valid = f"{ds_path}-files/{ds_name}-ft.valid.jsonl"

dataset_path_ft_train_v2 = f"{ds_path}-files/{ds_name}-ft.train.v2.jsonl"
dataset_path_ft_valid_v2 = f"{ds_path}-files/{ds_name}-ft.valid.v2.jsonl"

print(f"Reading arrow file {raft_arrow_file}")

Reading arrow file dataset/zava-articles/data-00000-of-00001.arrow


### Export dataset to JSONL

Let's export the Apache Arrow format file to JSONL, easier to manipulate

In [16]:
! python .gorilla/raft/format.py \
    --input $raft_arrow_file \
    --output $dataset_path_hf \
    --output-format hf

Generating train split: 462 examples [00:00, 20012.48 examples/s]
2025-08-26 22:08:04  INFO [    ] raft Dataset has 462 rows
2025-08-26 22:08:04  INFO [    ] raft Converting arrow file dataset/zava-articles/data-00000-of-00001.arrow to jsonl hf file dataset/zava-articles-files/zava-articles-hf.full.jsonl
Creating json from Arrow format: 100%|████████████| 1/1 [00:00<00:00, 24.78ba/s]


In [9]:
hf_full_df = pd.read_json(dataset_path_hf, lines=True)
print(f"Dataset {dataset_path_hf} has {len(hf_full_df)} records")
hf_full_df.head(5)

Dataset dataset/zava-articles-files/zava-articles-hf.full.jsonl has 462 records


,id,type,question,context,oracle_context,cot_answer,instruction
0,83117ec4-edf7-4e04-bf00-c916975ddef9,general,What type of paint should be prioritized for o...,{'sentences': [['I have used a few different k...,High-VOC paints release harmful chemicals that...,Step-by-step reasoning:\n\n1. The question ask...,<DOCUMENT>I have used a few different kinds of...
1,cd5adb64-7e0a-455d-974f-6714b4eacdb2,general,Why should low-VOC or zero-VOC paints be chose...,{'sentences': [['High-VOC paints release harmf...,High-VOC paints release harmful chemicals that...,"To answer the question, I first need to identi...",<DOCUMENT>High-VOC paints release harmful chem...
2,9381329e-50f8-4d54-b2cf-81a832b5df6b,general,What should you do with windows when working i...,{'sentences': [['Tell us in the section below....,"If you're indoors, like when working near your...",Step-by-step reasoning:\n\n1. The question ask...,<DOCUMENT>Tell us in the section below.</DOCUM...
3,adcf736f-8b59-4d40-b704-b63b8d67143f,general,Is priming a step that can be skipped when pre...,{'sentences': [['That time she didn't leave wi...,"If you're indoors, like when working near your...","To answer the question ""Is priming a step that...",<DOCUMENT>That time she didn't leave without i...
4,ddad0120-29c3-4700-8c3a-2c341d6f7055,general,Who is the current president of the United Sta...,"{'sentences': [['2.', 'She never could have fo...",2.,Step-by-step reasoning:\n\n1. The question ask...,<DOCUMENT>2.</DOCUMENT>\n<DOCUMENT>She never c...


## Let's look at a sample

In [10]:
from IPython.display import display, Markdown
from random import randint

sample_idx = 2#randint(0, len(hf_full_df) - 1)
sample = hf_full_df.iloc[sample_idx]
instruction_md = sample.instruction.replace("<DOCUMENT>", "`<DOCUMENT>`").replace("</DOCUMENT>", "`</DOCUMENT>`")
oracle_context_md = sample.oracle_context.replace("<DOCUMENT>", "`<DOCUMENT>`").replace("</DOCUMENT>", "`</DOCUMENT>`")
sample_answer_md = sample.cot_answer.replace("<ANSWER>", "`<ANSWER>`").replace("##begin_quote##", "`##begin_quote##`").replace("##end_quote##", "`##end_quote##`")
print(f"Showing sample of {dataset_path_hf}")
display(Markdown(f"""
## Oracle Context
{oracle_context_md}

## Instructions
{instruction_md}

## Question
{sample.question}

## CoT Answer
{sample_answer_md}
"""))

Showing sample of dataset/zava-articles-files/zava-articles-hf.full.jsonl



## Oracle Context
If you're indoors, like when working near your beloved Linda Sleigh Solid Wood Bed 
Espresso, ensure windows are open to let particles escape. 1.3 Applying Primer
Priming is a step you should never skip.

## Instructions
`<DOCUMENT>`Tell us in the section below.`</DOCUMENT>`
`<DOCUMENT>`Allow to dry completely before applying a topcoat. Ideas For Painting Garden Furniture
Looking for inspiration for your outdoor space? Here are some popular colour ideas and themes for painting your garden furniture:
- Gentle colour: mellow tones (e.g. light blues and pinks) can brighten a space without being overpowering. - Creative furniture: upcycle items and paint planters, benches and tables for a coordinated look. - Go full colour: pick a bold RAL/paint shade for a striking focal point in the garden. Frequently Asked Questions
Q. How do you repaint garden furniture? A. Remove peeling paint, prepare the surface, and use a paint or stain suitable for exterior use. Many recommended products are suitable for coating over 
previously painted surfaces.`</DOCUMENT>`
`<DOCUMENT>`She never could have found a set that she liked better if she were trying to buy new! What a great investment that table was for only $69! I found the placemats and florals at HomeGoods. The placemats were too big, but because they were plastic, we were able to make them fit by cutting a 
couple of inches off of each one. The pedestal base on this table was what sold me!`</DOCUMENT>`
`<DOCUMENT>`If you're indoors, like when working near your beloved Linda Sleigh Solid Wood Bed 
Espresso, ensure windows are open to let particles escape. 1.3 Applying Primer
Priming is a step you should never skip.`</DOCUMENT>`
What should you do with windows when working indoors near the Linda Sleigh Solid Wood Bed?  

## Question
What should you do with windows when working indoors near the Linda Sleigh Solid Wood Bed?  

## CoT Answer
Step-by-step reasoning:

1. The question asks what should be done with windows when working indoors near the Linda Sleigh Solid Wood Bed.
2. I need to look for any instructions or recommendations in the context about windows in this situation.
3. In the context, I find the following relevant sentence: `##begin_quote##`If you're indoors, like when working near your beloved Linda Sleigh Solid Wood Bed Espresso, ensure windows are open to let particles escape.`##end_quote##`
4. This sentence directly addresses the situation described in the question and provides a clear action to take regarding windows.
5. Therefore, the answer should be based on this instruction.

`<ANSWER>`: Ensure windows are open to let particles escape.


### Split the dataset into train / validation / evaluation

In machine learning, splitting a dataset into training, validation, and test sets is a fundamental step to ensure that your model is trained effectively, evaluated properly, and generalizes well to new data. Here’s a brief explanation of each split:

- **Training Split**: The training set (80% of the data) is used to train the model. It helps the model learn patterns and relationships by adjusting its internal parameters based on input-output pairs.

- **Validation Split**: The validation set (10%) is used during training to monitor the model’s performance and guide convergence. It helps fine-tune hyperparameters and ensures the model doesn’t overfit by providing feedback on unseen data during training.

- **Evaluation Split (sometimes also called test split)**: The test set (10%) is used only after training is complete to evaluate the model’s final performance. It provides an unbiased measure of how well the model generalizes to new, unseen data.

In [11]:
# split dataset into 80%/10%/10%
import numpy as np

samples_count = len(hf_full_df)
splits = [int(finetuning_train_split * samples_count), int((finetuning_train_split + finetuning_valid_split) * samples_count)]
print(f"Splitting dataset at {splits}")
hf_train_df, hf_valid_df, hf_eval_df = np.split(hf_full_df, splits)
hf_train_df.to_json(dataset_path_hf_train, orient="records", lines=True)
hf_valid_df.to_json(dataset_path_hf_valid, orient="records", lines=True)
hf_eval_df.to_json(dataset_path_hf_eval, orient="records", lines=True)

Splitting dataset at [369, 415]


/home/vscode/.local/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


### Export training and validation splits into JSONL format

In [12]:
! python .gorilla/raft/format.py \
    --input $dataset_path_hf_train \
    --input-type jsonl \
    --output $dataset_path_ft_train \
    --output-format $format \
    --output-completion-prompt-column text\
    --output-completion-completion-column ground_truth

Generating train split: 369 examples [00:00, 7498.54 examples/s]
2025-08-28 02:34:48  INFO [    ] raft Dataset has 369 rows
2025-08-28 02:34:48  INFO [    ] raft Converting jsonl file dataset/zava-articles-files/zava-articles-hf.train.jsonl to jsonl chat file dataset/zava-articles-files/zava-articles-ft.train.jsonl
Creating json from Arrow format: 100%|███████████| 1/1 [00:00<00:00, 107.89ba/s]


In [13]:
! python .gorilla/raft/format.py \
    --input $dataset_path_hf_valid \
    --input-type jsonl \
    --output $dataset_path_ft_valid \
    --output-format $format \
    --output-completion-prompt-column text\
    --output-completion-completion-column ground_truth

Generating train split: 46 examples [00:00, 2330.84 examples/s]
2025-08-28 02:34:51  INFO [    ] raft Dataset has 46 rows
2025-08-28 02:34:51  INFO [    ] raft Converting jsonl file dataset/zava-articles-files/zava-articles-hf.valid.jsonl to jsonl chat file dataset/zava-articles-files/zava-articles-ft.valid.jsonl
Creating json from Arrow format: 100%|███████████| 1/1 [00:00<00:00, 525.87ba/s]


In [14]:
dataset_path_ft_valid_df = pd.read_json(dataset_path_ft_valid, lines=True)
dataset_path_ft_valid_df.head(2)

,messages
0,[{'content': 'The following is a conversation ...
1,[{'content': 'The following is a conversation ...


### Keep the evaluation split aside

We don't need to format the evaluation dataset for now

In [15]:
pd.read_json(dataset_path_hf_eval, lines=True).head(2)

,id,type,question,context,oracle_context,cot_answer,instruction
0,eeb078a5-1503-46a0-971e-53ee533b9131,general,Are there furniture paints that are easier to ...,{'sentences': [['Easy DIY: How To Paint Outdoo...,There are many other furniture paints that are...,Step-by-step reasoning:\n\n1. The question ask...,<DOCUMENT>Easy DIY: How To Paint Outdoor Plast...
1,bac5929a-035d-4390-9177-e679345375ad,general,What technique helps create striking color com...,{'sentences': [['I was a bit surprised by this...,"When color coordination seems challenging, emp...","To answer the question, I first need to identi...",<DOCUMENT>I was a bit surprised by this conclu...


In [17]:
from lib.conversation_to_markdown import conversation_to_markdown
import json
# read first line of JSONL file as dict
# use json load
with open(dataset_path_ft_train, "r") as f:
    first_line = f.readline()
    # use json load
    first_record = json.loads(first_line)
    print("First record of " + dataset_path_ft_train)
    display(Markdown(conversation_to_markdown(first_record)))

First record of dataset/zava-articles-files/zava-articles-ft.train.jsonl


# AI Conversation

## `system`

The following is a conversation with an AI assistant. The assistant is helpful, clever, friendly and gives concise and accurate answers.

## `user`

`<DOCUMENT>`I have used a few different kinds of sprayers at all different price points, and all have been easy to use and have given 
an impeccable finish. Step 6: Sand and Wipe Down Again
KILZ 3® PREMIUM Primer is ready for paint in just one hour. After the primer has dried, lightly sand the surfaces once more to ensure a smooth finish.`</DOCUMENT>`
`<DOCUMENT>`High-VOC paints release harmful chemicals that affect people, plants, animals, 
and aquatic life during application and curing. Prioritize low-VOC or zero-VOC formulations for outdoor furniture projects. These environmentally responsible options protect your family's health while 
minimizing ecological impact.`</DOCUMENT>`
`<DOCUMENT>`Ensure adequate ventilation even in outdoor spaces, 
particularly when using oil-based products. Step 2: Thorough Furniture Cleaning 
Deep cleaning forms the foundation of professional results. For weathered outdoor furniture, use soap, water, and scrub brushes to remove accumulated 
dirt, grime, and surface contaminants. This process eliminates barriers that prevent proper paint adhesion.`</DOCUMENT>`
`<DOCUMENT>`Remember that outdoor-specific paints should remain outdoors.`</DOCUMENT>`
What type of paint should be prioritized for outdoor furniture projects?  

## `assistant`

Step-by-step reasoning:

1. The question asks what type of paint should be prioritized for outdoor furniture projects.
2. The context discusses the environmental and health impacts of different types of paint, specifically mentioning high-VOC paints and alternatives.
3. To answer, I need to identify which paint type is recommended in the context.
4. The context states: `##begin_quote##`High-VOC paints release harmful chemicals that affect people, plants, animals, and aquatic life during application and curing.`##end_quote##`
5. This means high-VOC paints are harmful and should be avoided.
6. The context further states: `##begin_quote##`Prioritize low-VOC or zero-VOC formulations for outdoor furniture projects. These environmentally responsible options protect your family's health while minimizing ecological impact.`##end_quote##`
7. Therefore, the recommended paint types are low-VOC or zero-VOC formulations.

`<ANSWER>`: Low-VOC or zero-VOC paint formulations should be prioritized for outdoor furniture projects.


# Reformat training dataset to more finetuning friendly format

In [12]:
from lib.reformat_conversation import reformat_ai_conversation
reformatted = reformat_ai_conversation(first_record)
print("Reformatted record:")
display(Markdown(conversation_to_markdown(reformatted)))


Reformatted record:


# AI Conversation

## `system`

The following is a conversation with an AI assistant. The assistant is helpful, clever, friendly and gives concise and accurate answers.

## `user`

Sources:
  - [1] I have used a few different kinds of sprayers at all different price points, and all have been easy to use and have given 
an impeccable finish. Step 6: Sand and Wipe Down Again
KILZ 3® PREMIUM Primer is ready for paint in just one hour. After the primer has dried, lightly sand the surfaces once more to ensure a smooth finish.
  - [2] High-VOC paints release harmful chemicals that affect people, plants, animals, 
and aquatic life during application and curing. Prioritize low-VOC or zero-VOC formulations for outdoor furniture projects. These environmentally responsible options protect your family's health while 
minimizing ecological impact.
  - [3] Ensure adequate ventilation even in outdoor spaces, 
particularly when using oil-based products. Step 2: Thorough Furniture Cleaning 
Deep cleaning forms the foundation of professional results. For weathered outdoor furniture, use soap, water, and scrub brushes to remove accumulated 
dirt, grime, and surface contaminants. This process eliminates barriers that prevent proper paint adhesion.
  - [4] Remember that outdoor-specific paints should remain outdoors.

What type of paint should be prioritized for outdoor furniture projects?

## `assistant`

Step-by-step reasoning:

1. The question asks what type of paint should be prioritized for outdoor furniture projects.
2. The context discusses the environmental and health impacts of different types of paint, specifically mentioning high-VOC paints and alternatives.
3. To answer, I need to identify which paint type is recommended in the context.
4. The context states: `High-VOC paints release harmful chemicals that affect people, plants, animals, and aquatic life during application and curing.` [2]
5. This means high-VOC paints are harmful and should be avoided.
6. The context further states: `Prioritize low-VOC or zero-VOC formulations for outdoor furniture projects. These environmentally responsible options protect your family's health while minimizing ecological impact.` [2]
7. Therefore, the recommended paint types are low-VOC or zero-VOC formulations.

`<ANSWER>`: Low-VOC or zero-VOC paint formulations should be prioritized for outdoor furniture projects.


In [ ]:
from lib.reformat_jsonl import reformat_jsonl_file
reformat_jsonl_file(dataset_path_ft_train, dataset_path_ft_train_v2)
reformat_jsonl_file(dataset_path_ft_valid, dataset_path_ft_valid_v2)

Reformatting complete:
  - Successfully reformatted: 369 conversations
  - Errors encountered: 0
  - Output written to: dataset/zava-articles-files/zava-articles-ft.train.v2.jsonl
Reformatting complete:
  - Successfully reformatted: 46 conversations
  - Errors encountered: 0
  - Output written to: dataset/zava-articles-files/zava-articles-ft.valid.v2.jsonl


## Next step -> Fine-tuning

- [./2_finetune_oai.ipynb](./2_finetune_oai.ipynb) if fine-tuning an **OpenAI** student model such as GPT-4o-mini
